
# Tic-Tac-Toe Neuroevolution Tournament - Notebook version 

Outline:
- Imports and dependencies
- Core exceptions and helpers
- Model, player, and game mechanics
- Match simulation and tournament orchestration
- Adherence to FAIR principles



## Imports and dependencies

Keras and TensorFlow must be available to construct and use the policy network.  
NumPy is used for array handling, Random for pairing and sampling, and JSON for optional weight export.


In [1]:
import keras
from keras import layers
import random
import numpy as np
import json


## IncorrectMoveError and basic validation helpers

Custom exception used to signal an illegal move. Illegal moves are detected when a player selects an occupied square.


In [2]:
class IncorrectMoveError(Exception):
    """Exception raised for an incorrect move in the game."""
    pass


## Vector to board position

Converts a one-hot 9-length vector into a `(row, col)` move on the 3x3 grid.  
- Validates length and one-hot structure.  
- Returns coordinates to be applied to the game board.


In [3]:
def vector_to_position(vector):
    # Convert input to a NumPy array (if it's not already)
    vector = np.array(vector)

    # Check that the vector has exactly 9 elements
    if vector.shape[0] != 9:
        raise ValueError("Input vector must have exactly 9 elements.")

    # Ensure that exactly one element is 1 and the rest are 0s
    if np.count_nonzero(vector == 1) != 1 or np.count_nonzero(vector != 0) != 1:
        raise ValueError("Input vector must be one-hot: exactly one element is 1 and the rest are 0.")

    # Find the index where the 1 is located
    index = int(np.where(vector == 1)[0][0])

    # Convert the 1D index to 2D coordinates in a 3x3 array
    row = index // 3
    col = index % 3

    return (row, col)


## Player

Player class to hold information for a single player, has a `select_move(state)` method which returns a one-hot length 9 vector, which is the player selection for the current move based on the board state:
- Mutation logic can perturb weights to create offspring for the next generation.  
- Move selection uses greedy argmax over the network output.


In [4]:
class Player:
    def __init__(self, genetics, mutation_rate, mutation_scale):
        # Create a new model with the same architecture
        self.model = keras.Sequential([
            layers.Input(shape=(9,)),
            layers.Dense(9, activation="relu", name="layer1"),
            layers.Dense(9, activation="relu", name="layer2"),
            layers.Dense(9, activation='softmax')
        ])

        if genetics is not None:
            mutated_weights = []
            for weight_array in genetics:
                # Create a mask that determines which elements will be mutated
                mutation_mask = np.random.rand(*weight_array.shape) < mutation_rate

                # Generate random Gaussian noise for mutations
                noise = np.random.normal(loc=0.0, scale=mutation_scale, size=weight_array.shape)

                # Apply mutations: only modify weights where the mask is True
                new_weight_array = weight_array + mutation_mask * noise
                mutated_weights.append(new_weight_array)



            # Set the mutated weights to the new model
            self.model.set_weights(mutated_weights)

    def select_move(self, state):
        move = self.model(state.reshape(1, 9))

        # Find the index of the highest value in the output
        best_index = np.argmax(move)

        # Create a one-hot encoded vector of the same shape as the output
        selected_move = np.zeros_like(move.flatten())
        selected_move[best_index] = 1
        return selected_move


## Single game simulation

Runs a head-to-head match between two players:
- Alternates turns until a win or a draw.  
- Tracks illegal moves and applies a penalty to fitness.  
- Returns the fitness scores for both players and basic bookkeeping such as turn count.


In [5]:
def single_game(player1, player2):
    no_turns = 0
    incorrect_counter = {0:0, 1:0}

    game = Game()
    while no_turns < 10:
        player1_move = player1.select_move(game.board)
        try:
            game.turn(1, player1_move)
        except IncorrectMoveError:
            incorrect_counter[0]+=1

        player2_move = player2.select_move(game.board)

        try:
            game.turn(2, player2_move)
        except IncorrectMoveError:
            incorrect_counter[1]+=1

        if game.evaluate() != 0:
            break

        no_turns += 1

    penalty_factor = 0.1

    outcome = game.evaluate()
    if outcome == 1:
        base_fitness1 = 1.0
        base_fitness2 = 0.0
    elif outcome == 2:
        base_fitness1 = 0.0
        base_fitness2 = 1.0
    else:
        base_fitness1 = 0.5
        base_fitness2 = 0.5

        # Subtract penalty for each illegal move:
    fitness1 = base_fitness1 - penalty_factor * incorrect_counter[0]
    fitness2 = base_fitness2 - penalty_factor * incorrect_counter[1]

    return fitness1, fitness2, incorrect_counter, no_turns, game.board


## Tournament orchestration

Handles tournament rounds, pairing, winner selection, and reproduction:
- `round()` pairs players, runs matches, and keeps winners.  
- `run_tournament()` executes rounds until a single winner remains.  
- `reproduce(...)` creates a new generation by sampling parents and applying mutation.


In [6]:
class Tournament:
    def __init__(self, no_players):
        self.no_players = no_players
        self.all_players = {}
        for i in range(no_players):
            self.all_players[i] = Player(None, None, None)

        self.remaining_ids = list(self.all_players.keys())

    def round(self):
        pairable_ids = self.remaining_ids

        player1_indices = [pairable_ids[i] for i in range(0, len(pairable_ids), 2)]
        player2_indices = [pairable_ids[i] for i in range(1, len(pairable_ids), 2)]

        winning_indices = []
        fitness_counter = 0

        assert len(player1_indices) == len(player2_indices)

        for match_no in range(len(player1_indices)):
            player1_index = player1_indices[match_no]
            player2_index = player2_indices[match_no]

            fitness1, fitness2, incorrect_counter, _, _ = single_game(
                self.all_players[player1_index], self.all_players[player2_index]
            )

            # Use the fitness scores to decide the winner.
            if fitness1 > fitness2:
                winning_indices.append(player1_index)
                fitness_counter += fitness1
                # print(f"Match {match_no}: Player 1 wins (fitness {fitness1:.2f} vs {fitness2:.2f}).")
            elif fitness2 > fitness1:
                winning_indices.append(player2_index)
                fitness_counter += fitness2
                # print(f"Match {match_no}: Player 2 wins (fitness {fitness2:.2f} vs {fitness1:.2f}).")
            else:
                winner = random.choice([player1_index, player2_index])
                winning_indices.append(winner)
                fitness_counter+=fitness1
                # print(f"Match {match_no}: Tie; randomly selected Player {1 if winner == player1_index else 2}.")

        print(f'Fitness: {fitness_counter/len(player1_indices)}')

        self.remaining_ids = winning_indices

        return winning_indices

    def run_tournament(self):
        round_number = 0
        winners = {}
        while len(self.remaining_ids) > 1:
            winners[round_number] = self.round()
            round_number += 1

        return winners

    def reproduce(self, winning_numbers, mutation_rate, mutation_scale):
        # Count wins for each player across all rounds.
        wins_count = {}
        for round_no, winners in winning_numbers.items():
            for player_index in winners:
                wins_count[player_index] = wins_count.get(player_index, 0) + 1

        # give even low performers a base chance (e.g., add 1 to every count).
        # This way, even players with 0 wins in the recorded rounds might get selected occasionally.
        for player_index in wins_count:
            wins_count[player_index] += 1

        # Build a reproduction pool where a player's index appears in proportion to its win count.
        reproduction_pool = []
        for player_index, count in wins_count.items():
            reproduction_pool.extend([player_index] * count)

        # Create a new generation of players.
        new_generation = {}
        for new_index in range(self.no_players):
            # Randomly select a parent from the reproduction pool.
            parent_index = random.choice(reproduction_pool)
            parent = self.all_players[parent_index]
            parent_weights = parent.model.get_weights()

            # Create a new Player with the parent's weights and apply mutation.
            # The Player __init__ handles the mutation when genetics are provided.
            new_generation[new_index] = Player(parent_weights, mutation_rate, mutation_scale)

        self.all_players = new_generation

        self.remaining_ids = list(self.all_players.keys())


## Game

Maintains a 3x3 board and provides:
- Row, column, and diagonal win checks.  
- `evaluate()` to determine win, draw, or ongoing game.  
- `turn(player, location_vec)` to place a mark, raising `IncorrectMoveError` for illegal moves.


In [7]:
class Game:
    def __init__(self):
        self.board = np.array([[0, 0, 0],
                               [0, 0, 0],
                               [0, 0, 0]])

    def row_win(self, player):
        for x in range(len(self.board)):
            win = True

            for y in range(len(self.board)):
                if self.board[x, y] != player:
                    win = False
                    continue

            if win == True:
                return (win)
        return (win)

    def col_win(self, player):
        for x in range(len(self.board)):
            win = True

            for y in range(len(self.board)):
                if self.board[y][x] != player:
                    win = False
                    continue

            if win == True:
                return (win)
        return (win)

    def diag_win(self, player):
        win = True
        y = 0
        for x in range(len(self.board)):
            if self.board[x, x] != player:
                win = False
        if win:
            return win
        win = True
        if win:
            for x in range(len(self.board)):
                y = len(self.board) - 1 - x
                if self.board[x, y] != player:
                    win = False
        return win

    def evaluate(self):
        winner = 0

        for player in [1, 2]:
            if (self.row_win(player) or
                    self.col_win(player) or
                    self.diag_win(player)):
                winner = player

        if np.all(self.board != 0) and winner == 0:
            winner = -1
        return winner

    def turn(self, player, location_vec):
        # Convert the vector to a (row, col) position
        position = vector_to_position(location_vec)
        row, col = position

        # Check if the move is valid (i.e., the cell is empty)
        if self.board[row][col] == 0:
            self.board[row][col] = player
        else:
            raise IncorrectMoveError(f"Invalid move! The cell at {position} is already taken.")


## Main experiment

Example run that sets hyperparameters, launches a tournament, and saves the top models.  
This section can be executed as-is to reproduce the behaviour from the script when dependencies are installed.


In [8]:
if __name__ == "__main__":
    mutation_rate = 0.01
    mutation_scale = 0.05
    test_tournament = Tournament(64)
    winning_numbers = test_tournament.run_tournament()

    for i in range(50):
        print(i)
        test_tournament.reproduce(winning_numbers, mutation_rate, mutation_scale)
        winning_numbers = test_tournament.run_tournament()


    def save_model_weights_to_json(model, file_path="weights.json"):
        # Get the weights from the model (a list of NumPy arrays)
        weights = model.get_weights()
        # Convert each NumPy array to a Python list so that it's JSON serializable
        weights_serializable = [w.tolist() for w in weights]

        # Write the list of weights to a JSON file
        with open(file_path, "w") as f:
            json.dump(weights_serializable, f, indent=2)


Fitness: -0.28125
Fitness: -0.1875
Fitness: -0.275
Fitness: -0.275
Fitness: -0.30000000000000004
Fitness: -0.30000000000000004
0
Fitness: -0.115625
Fitness: -0.19374999999999998
Fitness: -0.07500000000000005
Fitness: -0.325
Fitness: -0.25000000000000006
Fitness: -0.20000000000000007
1
Fitness: -0.20312500000000008
Fitness: -0.2687500000000001
Fitness: -0.2875
Fitness: -0.275
Fitness: -0.30000000000000004
Fitness: -0.30000000000000004
2
Fitness: -0.28125
Fitness: -0.26875000000000004
Fitness: -0.11250000000000004
Fitness: -0.20000000000000007
Fitness: -0.20000000000000007
Fitness: -0.30000000000000004
3
Fitness: -0.27187500000000003
Fitness: -0.29375
Fitness: -0.22500000000000006
Fitness: -0.20000000000000007
Fitness: -0.25000000000000006
Fitness: -0.30000000000000004
4
Fitness: -0.26875
Fitness: -0.26250000000000007
Fitness: -0.2500000000000001
Fitness: -0.25000000000000006
Fitness: -0.30000000000000004
Fitness: -0.30000000000000004
5
Fitness: -0.2531250000000001
Fitness: -0.1937500000

# Adherence to FAIR principles
- Findable: Has metadata and tags, is publicly available on GitHub
- Accessible: Code is retrievable for free from GitHub, with MIT licence
- Interoperable: Uses open formats (python, Jupyter, JSON)
- Reusable: Available under MIT licence, instructions given for running the code in ReadMe.
